# Integrazione CoreML di RusTorch - Binding Python

Questo notebook dimostra come utilizzare la funzionalità CoreML di RusTorch tramite i binding Python.

## Configurazione e Importazioni

In [ ]:
# Importare i binding Python di RusTorch
try:
    import rustorch
    print(f"✅ Versione RusTorch: {rustorch.__version__}")
    print(f"📝 Descrizione: {rustorch.__description__}")
    print(f"👥 Autore: {rustorch.__author__}")
except ImportError as e:
    print(f"❌ Importazione di RusTorch fallita: {e}")
    print("Si prega di compilare con maturin develop")
    exit(1)

import numpy as np
import platform

print(f"🖥️ Piattaforma: {platform.system()} {platform.release()}")
print(f"🐍 Versione Python: {platform.python_version()}")

## Verificare la Disponibilità di CoreML

In [ ]:
# Verificare la funzionalità CoreML
try:
    # Verificare se CoreML è disponibile
    coreml_available = rustorch.is_coreml_available()
    print(f"🍎 CoreML disponibile: {coreml_available}")
    
    if coreml_available:
        print("🎉 CoreML è disponibile!")
        
        # Ottenere informazioni sul dispositivo
        device_info = rustorch.get_coreml_device_info()
        print("📱 Informazioni dispositivo CoreML:")
        print(device_info)
    else:
        print("⚠️ CoreML non è disponibile")
        if platform.system() != "Darwin":
            print("CoreML è disponibile solo su macOS")
        else:
            print("Le funzionalità CoreML potrebbero non essere abilitate")
            
except AttributeError:
    print("❌ Funzioni CoreML non trovate")
    print("Potrebbe non essere compilato con funzionalità CoreML")
    coreml_available = False
except Exception as e:
    print(f"❌ Errore nella verifica di CoreML: {e}")
    coreml_available = False

## Creazione Dispositivo CoreML e Operazioni

In [ ]:
if coreml_available:
    try:
        # Creare dispositivo CoreML
        device = rustorch.CoreMLDevice(device_id=0)
        print(f"🖥️ Dispositivo CoreML creato: {device}")
        
        # Ottenere informazioni sul dispositivo
        print(f"🆔 ID dispositivo: {device.device_id()}")
        print(f"✅ Disponibile: {device.is_available()}")
        print(f"💾 Limite memoria: {device.memory_limit()} byte")
        print(f"🧮 Limite unità di calcolo: {device.compute_units_limit()}")
        print(f"📚 Dimensione cache modello: {device.model_cache_size()}")
        
        # Pulizia cache
        device.cleanup_cache()
        print("🧹 Cache pulita")
        
    except Exception as e:
        print(f"❌ Errore operazione dispositivo CoreML: {e}")
else:
    print("⚠️ Saltando operazioni dispositivo poiché CoreML non è disponibile")

## Configurazione Backend CoreML

In [ ]:
if coreml_available:
    try:
        # Creare configurazione backend CoreML
        config = rustorch.CoreMLBackendConfig(
            enable_caching=True,
            max_cache_size=200,
            enable_profiling=True,
            auto_fallback=True
        )
        print(f"⚙️ Configurazione backend: {config}")
        
        # Verificare e modificare valori di configurazione
        print(f"📊 Abilita caching: {config.enable_caching}")
        print(f"🗂️ Dimensione max cache: {config.max_cache_size}")
        print(f"📈 Abilita profiling: {config.enable_profiling}")
        print(f"🔄 Fallback automatico: {config.auto_fallback}")
        
        # Modificare configurazione
        config.enable_profiling = False
        config.max_cache_size = 150
        print(f"\n🔧 Configurazione aggiornata: {config}")
        
        # Creare backend CoreML
        backend = rustorch.CoreMLBackend(config)
        print(f"🚀 Backend CoreML: {backend}")
        print(f"✅ Backend disponibile: {backend.is_available()}")
        
        # Ottenere statistiche backend
        stats = backend.get_stats()
        print(f"📊 Statistiche backend: {stats}")
        print(f"   Operazioni totali: {stats.total_operations}")
        print(f"   Hit cache: {stats.cache_hits}")
        print(f"   Miss cache: {stats.cache_misses}")
        print(f"   Operazioni fallback: {stats.fallback_operations}")
        print(f"   Tasso hit cache: {stats.cache_hit_rate():.2%}")
        print(f"   Tasso fallback: {stats.fallback_rate():.2%}")
        print(f"   Tempo esecuzione medio: {stats.average_execution_time_ms:.2f}ms")
        
        # Pulizia cache
        backend.cleanup_cache()
        print("\n🧹 Cache backend pulita")
        
    except Exception as e:
        print(f"❌ Errore operazione backend CoreML: {e}")
else:
    print("⚠️ Saltando operazioni backend poiché CoreML non è disponibile")

## Operazioni Tensore di Base (CPU)

Per confrontare con CoreML, eseguiamo prima operazioni di base su CPU.

In [ ]:
try:
    # Creazione e operazioni tensore di base
    print("🧮 Operazioni tensore di base (CPU)")
    
    # Creare tensori da array NumPy (interfaccia semplificata)
    data_a = np.random.randn(2, 3).astype(np.float32)
    data_b = np.random.randn(3, 2).astype(np.float32)
    
    print(f"📐 Forma matrice A: {data_a.shape}")
    print(f"📐 Forma matrice B: {data_b.shape}")
    
    # Moltiplicazione matrici con NumPy (per confronto)
    numpy_result = np.matmul(data_a, data_b)
    print(f"✅ Forma risultato matmul NumPy: {numpy_result.shape}")
    print(f"📊 Risultato (primi elementi): {numpy_result.flatten()[:4]}")
    
    print("\n🚀 Operazioni CPU completate")
    
except Exception as e:
    print(f"❌ Errore operazione tensore: {e}")

## Simulazione Confronto Prestazioni

In [ ]:
import time

def benchmark_matrix_operations():
    """Confrontare prestazioni con diverse dimensioni matrici"""
    
    sizes = [(64, 64), (128, 128), (256, 256), (512, 512)]
    
    print("🏁 Confronto prestazioni:")
    print("Dimensione\t\tTempo CPU (ms)\tCoreML Atteso (ms)")
    print("-" * 58)
    
    for size in sizes:
        # Misurare tempo esecuzione CPU
        a = np.random.randn(*size).astype(np.float32)
        b = np.random.randn(size[1], size[0]).astype(np.float32)
        
        start_time = time.time()
        result = np.matmul(a, b)
        cpu_time = (time.time() - start_time) * 1000
        
        # Tempo CoreML atteso (ipotetico)
        # Nell'implementazione reale, usare misurazioni reali dal backend CoreML
        expected_coreml_time = cpu_time * 0.6  # Assunzione: CoreML è 40% più veloce
        
        print(f"{size[0]}x{size[1]}\t\t{cpu_time:.2f}\t\t{expected_coreml_time:.2f}")

benchmark_matrix_operations()

print("\n📝 Nota: I tempi CoreML sono ipotetici. I valori reali dipendono dall'implementazione specifica.")

## Simulazione Selezione Dispositivo

In [ ]:
def simulate_device_selection():
    """Simulare selezione intelligente dispositivo"""
    
    operations = [
        ("Moltiplicazione matrice piccola", (16, 16), "CPU"),
        ("Moltiplicazione matrice media", (128, 128), "Metal GPU"),
        ("Moltiplicazione matrice grande", (512, 512), "CoreML" if coreml_available else "Metal GPU"),
        ("Funzione attivazione", (32, 64, 128, 128), "Metal GPU"),
        ("Convoluzione piccola", (1, 3, 32, 32), "CPU"),
        ("Convoluzione grande", (16, 64, 224, 224), "CoreML" if coreml_available else "Metal GPU"),
        ("Operazioni numeri complessi", (128, 128), "Metal GPU"),  # CoreML non supportato
        ("Distribuzione statistica", (1000,), "CPU"),  # CoreML non supportato
    ]
    
    print("🎯 Simulazione selezione intelligente dispositivo:")
    print("Operazione\t\t\tForma Tensore\t\tDispositivo Selezionato")
    print("-" * 78)
    
    for name, shape, device in operations:
        shape_str = "x".join(map(str, shape))
        print(f"{name:<31}\t{shape_str:<15}\t{device}")
    
    print("\n📝 Logica selezione:")
    print("  • Operazioni piccole: CPU (evitare overhead)")
    print("  • Operazioni medie: Metal GPU (bilanciato)")
    print("  • Operazioni grandi: CoreML (ottimizzato)")
    print("  • Operazioni non supportate: fallback GPU/CPU")

simulate_device_selection()

## Esempio Pratico: Strato Semplice Rete Neurale

In [ ]:
def simulate_neural_network_layer():
    """Simulare strato rete neurale"""
    
    print("🧠 Simulazione strato rete neurale:")
    
    # Dimensione batch e configurazione strato
    batch_size = 32
    input_features = 784  # 28x28 MNIST
    hidden_features = 256
    output_features = 10  # 10 classi
    
    print(f"📊 Dimensione batch: {batch_size}")
    print(f"🔢 Feature input: {input_features}")
    print(f"🧮 Feature nascoste: {hidden_features}")
    print(f"🎯 Feature output: {output_features}")
    
    # Simulazione forward pass
    steps = [
        ("Input → Nascosto", f"({batch_size}, {input_features}) @ ({input_features}, {hidden_features})", "CoreML" if coreml_available else "Metal"),
        ("Attivazione ReLU", f"({batch_size}, {hidden_features})", "Metal"),
        ("Nascosto → Output", f"({batch_size}, {hidden_features}) @ ({hidden_features}, {output_features})", "CoreML" if coreml_available else "Metal"),
        ("Softmax", f"({batch_size}, {output_features})", "CPU"),
    ]
    
    print("\n🔄 Simulazione forward pass:")
    total_time = 0
    
    for step, shape, device in steps:
        # Tempo esecuzione virtuale (ms)
        if device == "CoreML":
            time_ms = np.random.uniform(0.5, 2.0)
        elif device == "Metal":
            time_ms = np.random.uniform(1.0, 3.0)
        else:  # CPU
            time_ms = np.random.uniform(0.2, 1.0)
        
        total_time += time_ms
        print(f"  {step:<15} {shape:<30} {device:<8} {time_ms:.2f}ms")
    
    print(f"\n⏱️ Tempo totale forward pass: {total_time:.2f}ms")
    print(f"🚀 Throughput stimato: {1000/total_time:.0f} batch/secondo")

simulate_neural_network_layer()

## Riepilogo e Prossimi Passi

In [ ]:
print("📋 Riepilogo Integrazione CoreML RusTorch:")
print()
print("✅ Elementi completati:")
print("  • Configurazione ambiente Jupyter")
print("  • Creazione kernel Rust e binding Python")
print("  • Verifica disponibilità CoreML")
print("  • Gestione dispositivi e configurazione")
print("  • Statistiche e profiling backend")
print("  • Selezione intelligente dispositivo")
print()
print("🚧 Sviluppo futuro:")
print("  • Implementazione reale operazioni CoreML")
print("  • Benchmarking prestazioni")
print("  • Più funzioni attivazione e tipi strato")
print("  • Miglioramenti gestione errori")
print("  • Ottimizzazione memoria")
print()
print("🎯 Prossimi passi raccomandati:")
print("  1. Caricare e testare modelli CoreML reali")
print("  2. Confrontare prestazioni Metal e CoreML")
print("  3. Testare con workflow deep learning reali")
print("  4. Valutare in ambiente produzione")

if coreml_available:
    print("\n🎉 Congratulazioni! CoreML è disponibile e tutte le funzionalità possono essere testate.")
else:
    print("\n⚠️ CoreML non è disponibile, ma le funzionalità di base stanno funzionando.")
    print("   Raccomandiamo di compilare con funzionalità CoreML abilitate su macOS.")